In [1]:
import pandas as pd
import numpy as np

In [2]:
import lsdb
from tape import Ensemble, ColumnMapper
import matplotlib.pyplot as plt
import dask
dask.config.set({'temporary_directory': '/data/epyc/users/fbcb/tmp'})

from dask.distributed import Client
client = Client(n_workers=10, threads_per_worker=10,
                memory_limit="60G")

client

/astro/users/fbcb/.local/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34147 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:34147/status,
Dashboard: http://127.0.0.1:34147/status,Workers: 10
Total threads: 100,Total memory: 558.79 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45004,Workers: 10
Dashboard: http://127.0.0.1:34147/status,Total threads: 100
Started: Just now,Total memory: 558.79 GiB
Comm: tcp://127.0.0.1:45119,Total threads: 10
Dashboard: http://127.0.0.1:38350/status,Memory: 55.88 GiB
Nanny: tcp://127.0.0.1:40862,


In [3]:
# Load from the small Ensemble
ens = Ensemble(client=client)
ens.from_ensemble("./ztf_small_ensemble")

In [4]:
len(ens.object)

892844

In [5]:
sec_to_day = 1 / (24 * 60 * 60)
max_dt_hc = 6 * 40 * sec_to_day

min_consec_obs = int((30 * 60) / 40)

In [6]:
# do the intervals using the already created masks and use ()

https://stackoverflow.com/questions/35091879/merge-two-arrays-vertically-to-array-of-tuples-using-numpy

In [7]:
def identify_hc_objects_band(mjd, band, flag, mag, magerr, band_name, 
                             max_dt = max_dt_hc, min_len = min_consec_obs):
    
    if (band.size > 0):
        
        arr_band_name = np.array([band_name])
        bands_obs = np.unique(band)
    
        if np.isin(arr_band_name, bands_obs, invert=True)[0]:
            return 0, 0, [[0, 0]]

        band_mask = ((band == band_name) & (flag == 0) & (magerr>0)  
                    & (~np.isnan(mjd)) & (~np.isnan(mag)) & (~np.isnan(magerr))) # remove nans!
    
        if np.sum(band_mask) == 0:
            return 0, 0, [[0, 0]]
    
        else:
            # sort time
            srt = mjd.argsort()
            mjd = mjd[srt]
            mag = mag[srt]
            magerr = magerr[srt]
            
            srt_mask = band_mask[srt]
            
            # select times of good obs in a specific band
            mjd_band = mjd[srt_mask]
            mag_band = mag[srt_mask]
            magerr_band = magerr[srt_mask]
            
            # calculate the number of observations
            num_obs = len(mjd_band)
    
            if num_obs < min_len:
                return 0, 0, [[0, 0]]
    
            else:
                # calculate the difference in mjd for the same band
                dt = np.diff(mjd_band)
                len_dt = num_obs - 1
        
                # create an array of indexes of delta t
                idx_dt = np.indices((len_dt,))[0]
    
                # create a mask indicating if dt > max_dt
                above_max_dt = (dt > max_dt)
        
                # select indexes that satisfy dt > max_dt
                idx_mask = idx_dt[above_max_dt]
            
                # all observations are hc observations
                if (len(idx_mask) == 0):
                    mjd_start_hc = np.min(mjd_band)
                    mjd_end_hc = np.max(mjd_band)
                    
                    # range of mjd of hc observations
                    hc_mjd_range = np.array([mjd_start_hc, mjd_end_hc])
                    num_hc_obs = np.array([num_obs])
                    num_hc_int = 1
                    
                    return num_hc_int, hc_mjd_range, num_hc_obs
                    
                else:
                    # hc observations at the beggining or at the end
                    idx_0 = idx_mask[0]
                    last_idx = idx_mask[-1]
                
                    first_obs_hc = (idx_0 >= min_len)
                    last_obs_hc = (len_dt - last_idx >= min_len)
                
                    # hc observations in the middle
                    idx_mask_diff = np.diff(idx_mask)
                    high_cadence_mask = (idx_mask_diff >= (min_len + 1))
                    center_obs_hc = (np.sum(high_cadence_mask) >= 1)
                    
                    if np.sum(first_obs_hc) + np.sum(last_obs_hc) + np.sum(high_cadence_mask) >= 1: 
                    
                        if center_obs_hc:
                            idx_start_hc = idx_mask[:-1][high_cadence_mask] + 1
                            idx_end_hc = idx_start_hc + idx_mask_diff[high_cadence_mask] - 1
                        
                            # central range of mjd of hc observations
                            mjd_start_hc = mjd_band[idx_start_hc]
                            mjd_end_hc = mjd_band[idx_end_hc]
                        
                            # return the range of mjd and the number of observations
                            central_hc_range = np.c_[mjd_start_hc, mjd_end_hc]
                            len_central_hc = idx_mask_diff[high_cadence_mask] - 1
                    
                            if first_obs_hc:
                                mjd_1st_hc_start = np.min(mjd_band)
                                mjd_1st_hc_end = mjd_band[idx_0 - 1]
                            
                                # 1st hc observations
                                first_hc_range = np.array([mjd_1st_hc_start, mjd_1st_hc_end])
                                len_first_hc = [idx_0] # because it starts with index 0
                            
                                # range of hc observations
                                first_and_central = np.vstack((first_hc_range, central_hc_range))
                                len_first_and_central_hc = np.append(len_first_hc, len_central_hc)
                            
                                if last_obs_hc:
                                    mjd_last_hc_start = mjd_band[last_idx + 1]
                                    mjd_last_hc_end = mjd_band[-1]
                                
                                    # last hc observation
                                    last_hc_range = np.array([mjd_last_hc_start, mjd_last_hc_end])
                                    len_last_hc = [len(mjd_band) - (last_idx + 1)]
                            
                                    # range of hc observations
                                    hc_mjd_range = np.vstack((first_and_central, last_hc_range))
                                    num_hc_obs = np.append(len_first_and_central_hc, len_last_hc)
                                    num_hc_int = len(num_hc_obs)
                                
                    
                                else:
                                    hc_mjd_range = first_and_central
                                    num_hc_obs = len_first_and_central_hc
                                    num_hc_int = len(num_hc_obs)

                        
                            elif last_obs_hc:
                            
                                mjd_last_hc_start = mjd_band[last_idx + 1]
                                mjd_last_hc_end = mjd_band[-1]
                            
                                # last range of hc observations
                                last_hc_range = np.array([mjd_last_hc_start, mjd_last_hc_end])
                                len_last_hc = [len(mjd_band) - (last_idx + 1)]
                                
                                # total range of hc observations
                                hc_mjd_range = np.vstack((central_hc_range, last_hc_range))
                                num_hc_obs = np.append(len_central_hc, len_last_hc)
                                num_hc_int = len(num_hc_obs)
                                

                            else:
                                hc_mjd_range = central_hc_range
                                num_hc_obs = len_central_hc
                                num_hc_int = len(num_hc_obs)
                                
                
                        elif first_obs_hc:
                            mjd_1st_hc_start = np.min(mjd_band)
                            mjd_1st_hc_end = mjd_band[idx_0 - 1]
                        
                            # 1st range of hc observations
                            first_hc_range = np.array([mjd_1st_hc_start, mjd_1st_hc_end])
                            len_first_hc = [idx_0]
                        
                            if last_obs_hc:
                                mjd_last_hc_start = mjd_band[last_idx + 1]
                                mjd_last_hc_end = mjd_band[-1]
                            
                                # last range of hc observations
                                last_hc_range = np.array([mjd_last_hc_start, mjd_last_hc_end])
                                len_last_hc = [len(mjd_band) - (last_idx + 1)]
                            
                                # total range of hc observations
                                hc_mjd_range = np.vstack((first_hc_range, last_hc_range))
                                num_hc_obs = np.append(len_first_hc, len_last_hc)
                                num_hc_int = len(num_hc_obs)
                            
                            
                            else:
                                hc_mjd_range = np.array([[mjd_1st_hc_start, mjd_1st_hc_end]])
                                num_hc_obs = len_first_hc
                                num_hc_int = len(num_hc_obs)
                            
                    
                        elif last_obs_hc:
                            mjd_last_hc_start = mjd_band[last_idx + 1]
                            mjd_last_hc_end = mjd_band[-1]
                        
                            # last range of hc observations
                            hc_mjd_range = np.array([[mjd_last_hc_start, mjd_last_hc_end]])
                            num_hc_obs = [len(mjd_band) - (last_idx + 1)]
                            num_hc_int = len(num_hc_obs)
                        
                        return num_hc_int, hc_mjd_range, num_hc_obs
                    

                    else:
                        
                        return 0, 0, [[0, 0]]

    else:
        
        return 0, 0, [[0, 0]]

In [8]:
# Define output columns
output_cols = ["hc_int_g", "range_hc_g", "nobs_hc_g", 
               "hc_int_r", "range_hc_r", "nobs_hc_r",
               "hc_int_i", "range_hc_i", "nobs_hc_i"]

# Define DataFrame with loc and scale as meta
my_meta = pd.DataFrame(columns=output_cols, dtype=float) #dtype was str if I didn't specify the type

# ** kwargs
def determine_hc(mjd, band, flag, mag, magerr): #**kwargs
    # determine if a given object has high cadence observations in any band
    
    hc_intervals_g, mjd_range_hc_g, num_obs_hc_g = identify_hc_objects_band(mjd, band, flag, mag, magerr, 'g')
    hc_intervals_r, mjd_range_hc_r, num_obs_hc_r = identify_hc_objects_band(mjd, band, flag, mag, magerr, 'r')
    hc_intervals_i, mjd_range_hc_i, num_obs_hc_i = identify_hc_objects_band(mjd, band, flag, mag, magerr, 'i')
    
    return pd.Series([hc_intervals_g, mjd_range_hc_g, num_obs_hc_g,
                      hc_intervals_r, mjd_range_hc_r, num_obs_hc_r,
                      hc_intervals_i, mjd_range_hc_i, num_obs_hc_i],index=output_cols)

In [9]:
calc = ens.batch(
    determine_hc,
    'mjd', 'band', 
    'catflags', 
    'mag', 'magerr', 
    meta=my_meta,
    use_map=True)

Using generated label, result_1, for a batch result.


In [10]:
ens.object.join(calc).update_ensemble()

In [11]:
ens.object.columns

Index(['ps1_objid', 'ra', 'dec', 'ps1_gMeanPSFMag', 'ps1_rMeanPSFMag',
       'ps1_iMeanPSFMag', 'nobs_g', 'nobs_r', 'nobs_i', 'mean_mag_g',
       'mean_mag_r', 'mean_mag_i', 'Norder', 'Dir', 'Npix', 'hc_int_g',
       'range_hc_g', 'nobs_hc_g', 'hc_int_r', 'range_hc_r', 'nobs_hc_r',
       'hc_int_i', 'range_hc_i', 'nobs_hc_i'],
      dtype='object')

In [12]:
len(ens.object.columns)

24

In [13]:
ens.source.columns

Index(['ps1_objid_object', 'ra_object', 'dec_object', 'ps1_gMeanPSFMag_object',
       'ps1_rMeanPSFMag_object', 'ps1_iMeanPSFMag_object', 'nobs_g_object',
       'nobs_r_object', 'nobs_i_object', 'mean_mag_g_object',
       'mean_mag_r_object', 'mean_mag_i_object', 'Norder_object', 'Dir_object',
       'Npix_object', 'ps1_objid', 'ra', 'dec', 'mjd', 'mag', 'magerr', 'band',
       'catflags'],
      dtype='object')

In [14]:
len(ens.source.columns)

23

In [15]:
# %%time
ens.object.compute()

,ps1_objid,ra,dec,ps1_gMeanPSFMag,ps1_rMeanPSFMag,ps1_iMeanPSFMag,nobs_g,nobs_r,nobs_i,mean_mag_g,...,Npix,hc_int_g,range_hc_g,nobs_hc_g,hc_int_r,range_hc_r,nobs_hc_r,hc_int_i,range_hc_i,nobs_hc_i
_hipscat_index,,,,,,,,,,,,,,,,,,,,,
8152498278085165056,78002788370141979,278.837036,-24.998601,18.848700,17.978800,17.582800,31,491,0,18.647357,...,7240,0,0,"[[0, 0]]",3,"[[58310.28173, 58310.32923], [58311.26937, 583...","[104, 128, 132]",0,0,"[[0, 0]]"
8152498278659784704,78002788346641330,278.834686,-24.999172,20.849501,19.402000,18.757000,0,437,0,NaN,...,7240,0,0,"[[0, 0]]",3,"[[58310.28173, 58310.32878], [58311.26937, 583...","[100, 122, 131]",0,0,"[[0, 0]]"
8152498279284736000,78002788356791383,278.835709,-24.999079,20.305901,19.506701,19.174801,1,286,0,20.536425,...,7240,0,0,"[[0, 0]]",3,"[[58310.28219, 58310.32878], [58311.26937, 583...","[61, 76, 96]",0,0,"[[0, 0]]"
8152498281625157632,78002788317030640,278.831726,-24.999731,16.700899,15.940500,15.661100,27,447,0,16.716135,...,7240,0,0,"[[0, 0]]",3,"[[58310.28173, 58310.32923], [58311.26937, 583...","[104, 128, 132]",0,0,"[[0, 0]]"
8152498283432902656,78002788285621634,278.828522,-24.998909,21.199900,20.353500,19.974600,0,32,0,NaN,...,7240,0,0,"[[0, 0]]",0,0,"[[0, 0]]",0,0,"[[0, 0]]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8158317531602354176,80392795227509564,279.522736,-23.000608,20.420500,19.565201,19.160999,0,269,0,NaN,...,7246,0,0,"[[0, 0]]",1,"[[58311.26939, 58311.305]]",[51],0,0,"[[0, 0]]"
8158317592482676736,80392795032007768,279.503189,-23.002059,20.154800,19.363701,19.023899,38,462,0,20.280116,...,7246,0,0,"[[0, 0]]",3,"[[58310.28221, 58310.32924], [58311.26984, 583...","[75, 98, 101]",0,0,"[[0, 0]]"
8158317593376063488,80392795040928963,279.504044,-23.001131,19.856300,19.109100,18.846399,61,574,0,20.012665,...,7246,0,0,"[[0, 0]]",3,"[[58310.28174, 58310.32924], [58311.26939, 583...","[94, 123, 118]",0,0,"[[0, 0]]"


In [16]:
#hc_objects = 
ens.object.query("hc_int_g > 0 or hc_int_r > 0 or hc_int_i > 0").update_ensemble()

## Define a function to filter Source by recalculating intervals -- adds an interval label flag to source table

In [19]:
def filter_hc_intervals(df):
    mjd = df["mjd"].to_numpy()
    band = df["band"].to_numpy()
    flag = df["catflags"].to_numpy()
    mag = df["mag"].to_numpy()
    magerr = df["magerr"].to_numpy()

    # Denotes interval groupings
    interval_labels = np.zeros_like(mjd, dtype=int)

    # Interval calculation -- do it again
    # Loop over each band and calculate intervals
    for i, band_label in enumerate(["g","r","i"]):
        
        num_intervals, intervals, obs_per_interval = identify_hc_objects_band(mjd, band, flag, mag, magerr, band_label) #[f"{band_label}_intervals"] # This returns a pandas # This returns a pandas series
        
        band_mask = ((band == band_label) & (flag == 0) & (magerr>0)  
                     & (~np.isnan(mjd)) & (~np.isnan(mag)) & (~np.isnan(magerr)))  # define an initial band mask
        
        
        # Filter based on intervals
        if num_intervals > 0:
            # sort time
            srt = mjd.argsort()
            mjd = mjd[srt]
            #mag = mag[srt]
            #magerr = magerr[srt]
            
            srt_mask = band_mask[srt]
            
            for j, interval in enumerate(np.atleast_2d(intervals)):
                 # Mask times based on the interval
                interval_mask = ((srt_mask) & (mjd >= interval[0]) & (mjd <= interval[1]))
                
                # Generate labels for the interval (unique per band-interval)
                interval_labels += ((100*i)+j) * interval_mask

    df = df.assign(interval_flag = interval_labels)
    
    df = df[interval_labels.astype(bool)]
    return df

In [20]:
source_subset = ens.source.groupby(ens._id_col, 
                                   group_keys=False).apply(lambda x: filter_hc_intervals(x), 
                                                           meta=ens.source._meta.assign(interval_flag=None))

In [22]:
source_subset.head(5, npartitions=-1)

,ps1_objid_object,ra_object,dec_object,ps1_gMeanPSFMag_object,ps1_rMeanPSFMag_object,ps1_iMeanPSFMag_object,nobs_g_object,nobs_r_object,nobs_i_object,mean_mag_g_object,...,Npix_object,ps1_objid,ra,dec,mjd,mag,magerr,band,catflags,interval_flag
_hipscat_index,,,,,,,,,,,,,,,,,,,,,
8152498278085165056,78002788370141979,278.837036,-24.998601,18.8487,17.9788,17.5828,31,491,0,18.647357,...,7240,78002788370141979,278.837036,-24.998601,58720.26101,17.633213,0.034561,r,0,100
8152498278085165056,78002788370141979,278.837036,-24.998601,18.8487,17.9788,17.5828,31,491,0,18.647357,...,7240,78002788370141979,278.837036,-24.998601,58726.19630,17.567810,0.033601,r,-1,100
8152498278085165056,78002788370141979,278.837036,-24.998601,18.8487,17.9788,17.5828,31,491,0,18.647357,...,7240,78002788370141979,278.837036,-24.998601,58721.22646,18.040178,0.042007,r,-32768,100
8152498278085165056,78002788370141979,278.837036,-24.998601,18.8487,17.9788,17.5828,31,491,0,18.647357,...,7240,78002788370141979,278.837036,-24.998601,58312.27745,17.728863,0.036074,r,0,100
8152498278085165056,78002788370141979,278.837036,-24.998601,18.8487,17.9788,17.5828,31,491,0,18.647357,...,7240,78002788370141979,278.837036,-24.998601,58730.21560,17.649914,0.034816,r,0,100
